In [2]:
import ast
from pickletools import dis

from langchain_classic.chains.question_answering.map_reduce_prompt import messages
from pygments.lexer import this
from sqlalchemy.util import immutabledict

import os
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict, Optional

load_dotenv()

True

In [3]:
# LLM API

In [4]:
class HelloAgentsLLM:
    """
    用于调用任何兼容OpenAI接口的服务
    """
    def __init__(self,
                 model: Optional[str]=None,
                 apiKey: Optional[str]=None,
                 baseUrl: Optional[str]=None,
                 timeout:Optional[int]=None):
        self.model = model or os.getenv("LLM_MODEL_ID")
        apiKey = apiKey or os.getenv("LLM_API_KEY")
        baseUrl = baseUrl or os.getenv("LLM_BASE_URL")
        timeout = timeout or int(os.getenv("LLM_TIMEOUT", 60))

        if not all([self.model, apiKey, baseUrl]):
            raise ValueError("模式ID、api key、url服务地址必须在.env中或者实例化时定义")

        self.client = OpenAI(
            api_key=apiKey,
            base_url=baseUrl,
            timeout=timeout
        )

    def think(self, messages:List[Dict[str, str]], temperature:float=0.0) -> Optional[str]:
        """调用大模型进行思考和回应
        :param messages:
        :param temperature:
        :return:
        """
        print(f"🧠 正在调用 {self.model} 模型...")
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=temperature,
                stream=True
            )
            # 处理流式响应
            print("✅ 大语言模型响应成功:")
            collected_content = []
            for chunk in response:
                content = chunk.choices[0].delta.content or ""
                print(content, end='', flush=True)
                collected_content.append(content)
            print()
            return "".join(collected_content)


        except Exception as e:
            print(f"❌ 调用LLM API时发生错误: {e}")
            return None

In [5]:
# TOOLS

In [6]:

def search(query: str) -> str:
    """
    一个基于SerpApi的实战网页搜索引擎工具。
    它会智能地解析搜索结果，优先返回直接答案或知识图谱信息。
    """
    print(f"🔍 正在执行 [SerpApi] 网页搜索: {query}")
    return "兴趣岛公司有12423名员工"
    try:
        api_key = os.getenv("SERPAPI_API_KEY")
        if not api_key:
            return "错误:SERPAPI_API_KEY 未在 .env 文件中配置。"

        params = {
            "engine": "google",
            "q": query,
            "api_key": api_key,
            "gl": "cn",  # 国家代码
            "hl": "zh-cn", # 语言代码
        }

        client = SerpApiClient(params)
        results = client.get_dict()

        # 智能解析:优先寻找最直接的答案
        if "answer_box_list" in results:
            return "\n".join(results["answer_box_list"])
        if "answer_box" in results and "answer" in results["answer_box"]:
            return results["answer_box"]["answer"]
        if "knowledge_graph" in results and "description" in results["knowledge_graph"]:
            return results["knowledge_graph"]["description"]
        if "organic_results" in results and results["organic_results"]:
            # 如果没有直接答案，则返回前三个有机结果的摘要
            snippets = [
                f"[{i+1}] {res.get('title', '')}\n{res.get('snippet', '')}"
                for i, res in enumerate(results["organic_results"][:3])
            ]
            return "\n\n".join(snippets)

        return f"对不起，没有找到关于 '{query}' 的信息。"

    except Exception as e:
        return f"搜索时发生错误: {e}"

In [7]:
# 构建工具执行器
from typing import Dict, Any

class ToolExecutor:
    """
    一个工具的执行器，负责管理和执行工具
    """
    def __init__(self):
        self.tools: Dict[str, Dict[str, Any]] = {}
        # {'funcnamr': {"description": description, "func": func}}

    def registerTool(self, name:str, description:str, func:callable):
        """向工具箱注册一个新工具"""
        if name in self.tools:
            print(f"警告：工具`{name}`已存在，将被覆盖。")
        self.tools[name] = {"description": description, "func": func}
        print(f"工具`{name}`已注册。")

    def getTool(self, name:str) -> callable:
        """根据工具名称"""
        return self.tools.get(name, {}).get('func')

    def getAvailableTools(self) -> str:
        """获取所有可用工具的格式化描述字符串"""
        return "\n".join([
            f'- {name}: {info['description']}'
            for name, info in self.tools.items()
        ])

In [8]:
# REACT

In [9]:
REACT_PROMPT_TEMPLATE = """
你是一个具备推理和行动功能的AI助手。你可以通过思考分析问题，然后调用合适的工具来获取信息，最终给出准确的答案。

## 可用工具如下：
{tools}

## 工作流程
请严格按照以下格式进行回应，每次只能执行一个步骤：

- Thought:分析问题，确定需要什么信息，制定研究策略。
- Action:判断是否需要调用外部工具，还是已经有足够的信息得到问题的结果

## 重要提醒
1. 每次回应必须包含Thought和Action两部分；
2. 工具调用的格式必须遵循：工具名[参数]；
3. 只有当你确信有足够信息回答问题时，才使用Finish
4. 如果工具返回的信息不够，继续使用其他工具或相同工具的不同参数

## 输出格式
- 如果需要调用指定工具：
Thought:...
Action:{{tool_name}}[{{tool_input}}]
- 如果能确定最终的答案：
Thought:...
Action:Finish[研究结论]

## 当前任务
**Question:** {question}

## 执行历史
{history}

现在开始你的推理和行动
"""

import re
from typing import Tuple

class ReActAgent:
    """
    ReAct(Reasoning and Acting) Agent
    核心：思考-行动-观察
    结合推理和行动的智能体，能够：
    1. 分析问题并制定计划
    2. 调用外部工具获取信息
    3. 基于观察结果进行推理
    4. 迭代执行直到得出最终答案
    """
    def __init__(self,
        llm_client:HelloAgentsLLM,
        tool_executor: ToolExecutor,
        max_steps: int=5
    ):
        self.llm_client = llm_client
        self.tool_executor = tool_executor
        self.max_steps = max_steps
        self.history = []

    def _parse_output(self, text:str) -> Tuple[Optional[str], Optional[str]]:
        """解析LLM的输出，提取Thought和Action"""
        thought_match = re.search(r"Thought:(.*)", text)
        action_match = re.search(r"Action:(.*)", text)
        thought = thought_match.group(1).strip() if thought_match else None
        action = action_match.group(1).strip() if action_match else None
        return thought, action

    def _parse_action(self, action_text: str) -> Tuple:
        """解析Action字符串，提取工具名称和输入"""
        _match = re.match(r"(\w+)\[(.*)\]", action_text)
        if _match:
            return _match.group(1), _match.group(2)

        return None, None

    def _parse_action_input(self, action_text: str) -> str:
        """解析行动输入"""
        match = re.match(r"\w+\[(.*)\]", action_text)
        return match.group(1) if match else ""


    def run(self, question:str):
        """运行ReAct智能体来回答一个问题"""
        self.history = []     # 每次运行都重置历史记录
        current_step = 0

        print(f"\n🤖 开始处理问题: {question}")

        while current_step < self.max_steps:
            current_step += 1
            print(f"--- 第 {current_step} 步 ---")

            # 1. 格式化提示词
            tools_desc = self.tool_executor.getAvailableTools()
            history_str = '\n'.join(self.history)
            prompt = REACT_PROMPT_TEMPLATE.format(
                tools=tools_desc,
                question=question,
                history=history_str
            )
            # 2. 调用LLM进行思考
            messages = [{'role': 'user', 'content': prompt}]
            response_text = self.llm_client.think(messages)

            if not response_text:
                print("错误：LLM未正常响应。")
                break

            thought, action = self._parse_output(response_text)
            if thought:
                print(f"思考：{thought}\n行动：{action}")
            if not action:
                print(f"未能解析出有效Action,流程终止。:{response_text}")
                break

            # 4. 执行Action
            if action.startswith("Finish"):
                # 提取出最终答案并结束
                final_answer = self._parse_action_input(action)
                print(f"🎉 最终答案: {final_answer}")
                return final_answer

            tool_name, tool_input = self._parse_action(action)
            if not tool_name or tool_input is None:
                print("无效Action")
                continue

            print(f"🎬 行动: {tool_name}[{tool_input}]")

            tool_function = self.tool_executor.getTool(tool_name)
            if not tool_function:
                observation = f"错误:未找到名为 '{tool_name}' 的工具。"
            else:
                observation = tool_function(tool_input) # 调用真实工具

            print(f"👀 观察: {observation}")

            # 将本轮的Action和Observation添加到历史记录中
            self.history.append(f"Action: {action}")
            self.history.append(f"Observation: {observation}")

        # 循环结束
        print("已达到最大步数，流程终止。")
        return None

In [10]:
llm = HelloAgentsLLM()
tool_executor = ToolExecutor()
tool_executor.registerTool(search.__name__, search.__doc__, search)
react_chain = ReActAgent(llm, tool_executor)
react_chain.run("搜索一下兴趣岛有多少员工")

工具`search`已注册。

🤖 开始处理问题: 搜索一下兴趣岛有多少员工
--- 第 1 步 ---
🧠 正在调用 doubao-seed-1-6-251015 模型...
✅ 大语言模型响应成功:
Thought:用户想知道兴趣岛有多少员工，首先需要明确“兴趣岛”是否为某个公司或组织的名称，然后查找其公开的员工数量信息。这类数据通常可能在公司官网、招聘页面、企业信息平台或新闻报道中提及，因此需要通过搜索引擎获取相关结果。
Action:search[兴趣岛 员工数量]
思考：用户想知道兴趣岛有多少员工，首先需要明确“兴趣岛”是否为某个公司或组织的名称，然后查找其公开的员工数量信息。这类数据通常可能在公司官网、招聘页面、企业信息平台或新闻报道中提及，因此需要通过搜索引擎获取相关结果。
行动：search[兴趣岛 员工数量]
🎬 行动: search[兴趣岛 员工数量]
🔍 正在执行 [SerpApi] 网页搜索: 兴趣岛 员工数量
👀 观察: 兴趣岛公司有12423名员工
--- 第 2 步 ---
🧠 正在调用 doubao-seed-1-6-251015 模型...
✅ 大语言模型响应成功:
Thought:已通过搜索获取到兴趣岛公司员工数量为12423名，信息明确且满足用户需求，无需进一步搜索。
Action:Finish[兴趣岛公司有12423名员工]
思考：已通过搜索获取到兴趣岛公司员工数量为12423名，信息明确且满足用户需求，无需进一步搜索。
行动：Finish[兴趣岛公司有12423名员工]
🎉 最终答案: 兴趣岛公司有12423名员工


'兴趣岛公司有12423名员工'

In [11]:
# Plan-and-Solve
# 分为规划器和执行器

In [12]:
PLANNER_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。你的任务是将用户提出的复杂问题分解成一个由多个简单步骤组成的行动计划。
请确保计划中的每个步骤都是独立的、可执行的子任务，并且严格按照逻辑顺序排列。
你的输出必须是一个python列表，其中每个元素都是一个描述子任务的字符串。

问题：{question}

请严格按照以下格式输出你的计划，```python与```作为前后缀是必要的：
```python
["步骤1", "步骤2", "步骤3", ...]
```
"""

EXECUTOR_PROMPT_TEMPLATE = """
你是以为顶级的AI执行专家。你的任务是严格的按照给定的计划，一步步地解决问题。
你将收到原始问题、完整的计划、以及到目前为止已经完成的步骤和结果。
请你专注于解决“当前步骤”，并仅输出该步骤的最终答案，不要输出任何额外的解释或对话。

# 原始问题：
{question}

# 完整计划：
{plan}

# 历史步骤与结果
{history}

# 当前步骤：
{current_step}

请仅输出针对“当前步骤”的回答：
"""

class Planner:
    def __init__(self, llm:HelloAgentsLLM):
        self.llm = llm

    def plan(self, question:str) -> list[str]:
        """根据用户的问题生成一个行动计划"""
        prompt = PLANNER_PROMPT_TEMPLATE.format(question=question)
        messages = [{'role': 'user', 'content': prompt}]

        print("--- 正在生成计划 ---")
        response_text = self.llm.think(messages) or ""
        print(f"✅ 计划已生成:\n{response_text}")

        try:
            plan_str = response_text.split("```python")[1].split("```")[0].strip()
            plan = ast.literal_eval(plan_str)
            return plan if isinstance(plan, list) else []
        except (ValueError, SyntaxError, IndexError) as e:
            print(f"❌ 解析计划时出错: {e}")
            print(f"原始响应: {response_text}")
            return []
        except Exception as e:
            print(f"❌ 解析计划时发生未知错误: {e}")
            return []

class Executor:
    def __init__(self, llm:HelloAgentsLLM):
        self.llm = llm

    def execute(self, question:str, plan:list[str]) -> str:
        """根据计划，逐步执行并解决"""
        history = ""  # 用于存储历史步骤和结果
        print("\n--- 正在执行计划 ---")

        response_text = ""
        for i, step in enumerate(plan):
            print(f"\n-> 正在执行步骤 {i+1}/{len(plan)}:{step}")
            prompt = EXECUTOR_PROMPT_TEMPLATE.format(
                question=question,
                plan=plan,
                history=history if history else "无",
                current_step=step
            )
            messages = [{'role': 'user', 'content': prompt}]
            response_text = self.llm.think(messages) or ""
            history += f"步骤{i+1}: {step}\n结果：{response_text}\n\n"
            print(f"✅ 步骤 {i+1} 已完成，结果: {response_text}")

        final_answer = response_text
        return final_answer

class PlanAndSolveAgent:
    def __init__(self, llm:HelloAgentsLLM):
        """链接规划器和执行器：先规划，再执行"""
        self.llm = llm
        self.planner = Planner(llm)
        self.executor = Executor(llm)

    def run(self, question:str):
        print(f"\n--- 开始处理问题 ---\n问题：{question}")
        # 1. 调用规划器生成计划
        plans = self.planner.plan(question)

        if not plans:
            print(f"\n--- 任务终止 ---\n无法生成有效的行动计划")
            return
        # 2. 调用执行器执行计划
        final_answer = self.executor.execute(question, plans)

        print(f"\n--- 任务完成 ---\n最终答案：{final_answer}")


In [13]:
question = "一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？"
plan_and_solver = PlanAndSolveAgent(llm)
plan_and_solver.run(question)


--- 开始处理问题 ---
问题：一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？
--- 正在生成计划 ---
🧠 正在调用 doubao-seed-1-6-251015 模型...
✅ 大语言模型响应成功:
```python
["确定周一卖出的苹果数量为15个", "计算周二卖出的苹果数量：15×2", "计算周三卖出的苹果数量：周二的数量-5", "将三天卖出的苹果数量相加得到总数"]
```
✅ 计划已生成:
```python
["确定周一卖出的苹果数量为15个", "计算周二卖出的苹果数量：15×2", "计算周三卖出的苹果数量：周二的数量-5", "将三天卖出的苹果数量相加得到总数"]
```

--- 正在执行计划 ---

-> 正在执行步骤 1/4:确定周一卖出的苹果数量为15个
🧠 正在调用 doubao-seed-1-6-251015 模型...
✅ 大语言模型响应成功:
周一卖出的苹果数量为15个。
✅ 步骤 1 已完成，结果: 周一卖出的苹果数量为15个。

-> 正在执行步骤 2/4:计算周二卖出的苹果数量：15×2
🧠 正在调用 doubao-seed-1-6-251015 模型...
✅ 大语言模型响应成功:
周二卖出的苹果数量为30个。
✅ 步骤 2 已完成，结果: 周二卖出的苹果数量为30个。

-> 正在执行步骤 3/4:计算周三卖出的苹果数量：周二的数量-5
🧠 正在调用 doubao-seed-1-6-251015 模型...
✅ 大语言模型响应成功:
周三卖出的苹果数量为25个。
✅ 步骤 3 已完成，结果: 周三卖出的苹果数量为25个。

-> 正在执行步骤 4/4:将三天卖出的苹果数量相加得到总数
🧠 正在调用 doubao-seed-1-6-251015 模型...
✅ 大语言模型响应成功:
70个苹果。
✅ 步骤 4 已完成，结果: 70个苹果。

--- 任务完成 ---
最终答案：70个苹果。


In [14]:
# Reflection
# 执行 -> 反思(反馈) -> 优化
# 执行：可以是react或者plan-and-solve

In [19]:
from typing import List, Dict, Any, Optional

class Memory:
    """记忆模块，用于存储智能体的行动和反思"""
    def __init__(self):
        self.records: List[Dict[str, Any]] = []

    def add_record(self, record_type:str, content:str):
        """向记忆添加一条记录
        - record_type(str): 记录的类型（execution or reflection）
        - content(str): 记录的内容
        """
        record = {'type': record_type, 'content': content}
        self.records.append(record)
        print(f"📝 记忆已更新，新增一条 '{record_type}' 记录。")

    def get_trajectory(self) -> str:
        """将所有记忆记录格式化为一个连贯的字符串文本，用于构建提示词"""
        trajectory_parts = []
        for record in self.records:
            if record['type'] == 'execution':
                trajectory_parts.append(f'--- 上一轮尝试（代码）---\n{record['content']}')
            elif record['type'] == 'reflection':
                trajectory_parts.append(f'--- 评审员反馈 ---\n{record['content']}')

        return '\n\n'.join(trajectory_parts)


    def get_last_execution(self) -> Optional[str]:
        """获取最近一次的执行结果（例如，最新生成的代码）"""
        for record in reversed(self.records):
            if record['type'] == 'execution':
                return record['content']
        return None

# 初始执行提示词
INITIAL_PROMPT_TEMPLATE = """
你是一位资深的Python程序员。请根据以下要求，编写一个Python函数。
你的代码必须包含完整的函数签名、文档字符串，并遵循PEP 8编码规范。

要求: {task}

请直接输出代码，不要包含任何额外的解释。
"""

# 反思提示词
REFLECT_PROMPT_TEMPLATE = """
你是一位极其严格的代码评审专家和资深算法工程师，对代码的性能有极致的要求。
你的任务是审查以下Python代码，并专注于找出其在<strong>算法效率</strong>上的主要瓶颈。

# 原始任务:
{task}

# 待审查的代码:
```python
{code}
```

请分析该代码的时间复杂度，并思考是否存在一种<strong>算法上更优</strong>的解决方案来显著提升性能。
如果存在，请清晰地指出当前算法的不足，并提出具体的、可行的改进算法建议（例如，使用筛法替代试除法）。
如果代码在算法层面已经达到最优，才能回答“无需改进”。

请直接输出你的反馈，不要包含任何额外的解释。
"""

# 优化提示词
REFINE_PROMPT_TEMPLATE = """
你是一位资深的Python程序员。你正在根据一位代码评审专家的反馈来优化你的代码。

# 原始任务:
{task}

# 你上一轮尝试的代码:
{last_code_attempt}
评审员的反馈：
{feedback}

请根据评审员的反馈，生成一个优化后的新版本代码。
你的代码必须包含完整的函数签名、文档字符串，并遵循PEP 8编码规范。
请直接输出优化后的代码，不要包含任何额外的解释。
"""

class ReflectionAgent:
    def __init__(self, llm:HelloAgentsLLM, max_iterations=3):
        self.llm = llm
        self.memory = Memory()
        self.max_iterations = max_iterations

    def _get_llm_response(self, prompt:str) -> str:
        messages = [{'role': 'user', 'content':prompt}]
        response_text = self.llm.think(messages) or ""
        return response_text

    def run(self, task:str):
        print(f"\n--- 开始处理任务 ---\n任务: {task}")

        # --- 1. 初始执行 ---
        print("\n--- 正在进行初始尝试 ---")
        initial_prompt = INITIAL_PROMPT_TEMPLATE.format(task=task)
        initial_code = self._get_llm_response(initial_prompt)
        self.memory.add_record("execution", initial_code)

        # --- 2. 迭代循环:反思与优化 ---
        for i in range(self.max_iterations):
            print(f"\n--- 第 {i+1}/{self.max_iterations} 轮迭代 ---")

            # a. 反思
            print("\n-> 正在进行反思...")
            last_code = self.memory.get_last_execution()
            reflect_prompt = REFLECT_PROMPT_TEMPLATE.format(task=task, code=last_code)
            feedback = self._get_llm_response(reflect_prompt)
            self.memory.add_record("reflection", feedback)

            # b. 检查是否需要停止
            if "无需改进" in feedback:
                print("\n✅ 反思认为代码已无需改进，任务完成。")
                break

            # c. 优化
            print("\n-> 正在进行优化...")
            refine_prompt = REFINE_PROMPT_TEMPLATE.format(
                task=task,
                last_code_attempt=last_code,
                feedback=feedback
            )
            refined_code = self._get_llm_response(refine_prompt)
            self.memory.add_record("execution", refined_code)

        final_code = self.memory.get_last_execution()
        print(f"\n--- 任务完成 ---\n最终生成的代码:\n```python\n{final_code}\n```")
        return final_code

In [20]:
ref = ReflectionAgent(llm)
ref.run("编写一个Python函数，找出1到n之间所有的素数 (prime numbers)")


--- 开始处理任务 ---
任务: 编写一个Python函数，找出1到n之间所有的素数 (prime numbers)

--- 正在进行初始尝试 ---
🧠 正在调用 doubao-seed-1-6-251015 模型...
✅ 大语言模型响应成功:
def find_primes(n):
    """Find all prime numbers between 1 and n (inclusive).
    
    Args:
        n (int): The upper limit of the range to check for primes.
        
    Returns:
        list: A list of prime numbers from 1 to n.
        
    Examples:
        >>> find_primes(10)
        [2, 3, 5, 7]
        >>> find_primes(2)
        [2]
        >>> find_primes(1)
        []
    """
    if n < 2:
        return []
    
    sieve = [True] * (n + 1)
    sieve[0] = sieve[1] = False
    
    for i in range(2, int(n ** 0.5) + 1):
        if sieve[i]:
            sieve[i*i : n+1 : i] = [False] * len(sieve[i*i : n+1 : i])
    
    return [i for i, is_prime in enumerate(sieve) if is_prime]
📝 记忆已更新，新增一条 'execution' 记录。

--- 第 1/3 轮迭代 ---

-> 正在进行反思...
🧠 正在调用 doubao-seed-1-6-251015 模型...
✅ 大语言模型响应成功:
Time complexity: O(n log log n). The code uses the Sieve of Erat

'def find_primes(n):\n    """Find all prime numbers between 1 and n (inclusive).\n    \n    Args:\n        n (int): The upper limit of the range to check for primes.\n        \n    Returns:\n        list: A list of prime numbers from 1 to n.\n        \n    Examples:\n        >>> find_primes(10)\n        [2, 3, 5, 7]\n        >>> find_primes(2)\n        [2]\n        >>> find_primes(1)\n        []\n    """\n    if n < 2:\n        return []\n    \n    sieve = [True] * (n + 1)\n    sieve[0] = sieve[1] = False\n    \n    for i in range(2, int(n ** 0.5) + 1):\n        if sieve[i]:\n            sieve[i*i : n+1 : i] = [False] * len(sieve[i*i : n+1 : i])\n    \n    return [i for i, is_prime in enumerate(sieve) if is_prime]'

In [21]:
"""
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIEvUngAtMZddQSLA0qXIshFrs7KPGptrimbJ9SNlK6x3 lzl_xyc@163.com
"""

'\nssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIEvUngAtMZddQSLA0qXIshFrs7KPGptrimbJ9SNlK6x3 lzl_xyc@163.com\n'